<a href="https://colab.research.google.com/github/berkaykarlik/whoIsThatPokemon_CNN/blob/master/pokemon_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount the Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Import Libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
from tqdm import tqdm



Get the Images

In [0]:
path='/content/gdrive/My Drive/data/PokemonData'

classes = []
data = []
SIZE = 100

def get_data():
  class_no = 0
  for dir in tqdm(os.listdir(path)):
    classes.append(dir)
    dir_path = os.path.join(path,dir)
    for img in tqdm(os.listdir(dir_path)):
      try:
        img_array = cv2.imread(os.path.join(dir_path,img))
        resized_array = cv2.resize(img_array,(SIZE,SIZE))
        data.append([resized_array,class_no])
      except:
        pass
    class_no += 1
get_data()

Shuffle the Data

In [0]:
import random

random.shuffle(data)

for sample in data[:10]:
    print(classes[sample[1]])

Seperate features& labels, convert feature set to numpy array



In [0]:
x = []
y = []
for features,label in data:
    x.append(features)
    y.append(label)

x = np.array(x).reshape(-1, SIZE, SIZE, 3)
y = np.array(y)

Split Data

In [0]:
train_set_size = (int)((len(x)*8)/10)
test_set_size = len(x) - train_set_size

train_images, test_images = x[:train_set_size],x[train_set_size:]
train_labels, test_labels = y[:train_set_size],y[train_set_size:]

print(train_images.shape,test_images.shape)

CNN

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [0]:
train_images, test_images = train_images / 255.0, test_images / 255.0

In [0]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(SIZE, SIZE ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(150)
])

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=100, 
                    validation_data=(test_images, test_labels))